In [1]:
#import data_helpers2 as data_helpers
import numpy as np

import tensorflow as tf

# Parameters
# ==================================================

# Eval Parameters
'''
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_string(
    "checkpoint_dir", "", "Checkpoint directory from training run")
tf.flags.DEFINE_string(
    "vocab_file", "", "Vocabulary mapping used for training run")


# Misc Parameters
tf.flags.DEFINE_boolean(
    "allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean(
    "log_device_placement", False, "Log placement of ops on devices")


FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")
print("Loading data...")
x_test, y_test, vocabulary, vocabulary_inv = data_helpers.load_data(
    eval=True, vocab_file=FLAGS.vocab_file,
    cat1="./data/subreddit_news", cat2="./data/subreddit_aww")
y_test = np.argmax(y_test, axis=1)
print("Vocabulary size: {:d}".format(len(vocabulary)))
print("Test set size {:d}".format(len(y_test)))
'''
# Load data. Load your own data here

import csv
import numpy as np
samples = {}
ALPHABET = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+ =<>()[]{}"
FEATURE_LEN =1014
cdict = {}
for i,c in enumerate(ALPHABET):
    cdict[c] = i + 2
samples = {}
with open('/home/zhangjinbin/cnn_very_depp_29_nlp/dbpedia_data/test.csv') as f:
    reader = csv.DictReader(f,fieldnames=['class'],restkey='fields')
    for row in reader:
        label = row['class']
        if label not in samples:
            samples[label] = []
        sample = np.ones(FEATURE_LEN)
        count = 0
        for field in row['fields']:
            for char in field.lower():
                if char in cdict:
                    sample[count] = cdict[char]
                    count += 1
                if count >= FEATURE_LEN-1:
                    break
        samples[label].append(sample)
    samples_per_class = None
    classes = samples.keys()
    class_samples = []
    for c in classes:
        if samples_per_class is None:
            samples_per_class = len(samples[c])
        else:
            assert samples_per_class == len(samples[c])
        class_samples.append(samples[c])


In [2]:
def build_onehot(input_):
    target = np.zeros(len(classes))
    target[input_] = 1
    return target
y= []
y_ = []
for i in range(len(classes)):
    for j in range(samples_per_class):
        target =build_onehot(i)
        y.append(target)
        y_.append(i)
y_ = np.array(y_)

In [3]:
x = np.reshape(class_samples,(-1,FEATURE_LEN))
y = np.reshape(y,(-1,len(classes)))

In [4]:
print("\nEvaluating...\n")
# Evaluation
# ==================================================
checkpoint_dir = '/home/zhangjinbin/cnn_very_depp_29_nlp/runs_new/1468942578/checkpoints/'
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
print("checkpoint file: {}".format(checkpoint_file))


Evaluating...

checkpoint file: /home/zhangjinbin/cnn_very_depp_29_nlp/runs_new/1468942578/checkpoints/model-84000


In [5]:
import os
import time
import datetime
def batch_iter(x, y, batch_size, num_epochs, shuffle=False):
    """
    Generates a batch iterator for a dataset.
    """
    # data = np.array(data)
    data_size = len(x)
    num_batches_per_epoch = int(data_size/batch_size) + 1
    print("num batches per epoch is: " + str(num_batches_per_epoch))
    # Shuffle the data at each epoch
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        x_shuffled = x[shuffle_indices]
        y_shuffled = y[shuffle_indices]
    else:
        x_shuffled = x
        y_shuffled = y
    for batch_num in range(num_batches_per_epoch):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        x_batch = x_shuffled[start_index:end_index]
        y_batch = y_shuffled[start_index:end_index]
        batch = list(zip(x_batch, y_batch))
        yield batch

In [6]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name(
            "dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name(
            "fc-3/predictions").outputs[0]

        # Generate batches for one epoch
       
        batches = batch_iter(x, y,128, 1)

        # Collect the predictions here
        all_predictions = []

        for test_batch in batches:
            x_batch, y_batch = zip(*test_batch)
            batch_predictions = sess.run(
                predictions, {input_x: x_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate(
                [all_predictions, batch_predictions])

# Print accuracy
correct_predictions = float(sum(np.equal(all_predictions,y_)))
print("Total number of test examples: {}".format(len(y_)))
print("Accuracy: {:g}".format(correct_predictions/float(len(y_))))

num batches per epoch is: 547
Total number of test examples: 70000
Accuracy: 0.978257
